#LLM-NodeJS-50K Dataset generator

**Description:** This script can be used with openrouter.ai to generate JavaScript code samples from various LLMs.

**Insturctions:** In STEP 0, all required dependencies are installed. In STEP 1, we define the programming task templates. In STEP 2, helper functions are introduced. Finally, in STEP 3, the dataset is actually generated.

###STEP 0 -  Install all necessary dependencies

In [1]:
# STEP 0: Set up Node.js and verify the syntax of a simple JavaScript snippet
print("[*] Installing Node.js...")
!apt-get install -y nodejs npm >/dev/null 2>&1
print("[*] Installation: DONE")
print("[*] Testing syntax check with a trivial ES6 JavaScript file...")
!echo "const test = () => { console.log('NodeJS works'); }; test();" > working_Javascript_example.js
!echo "const test = () => { console.log(()'NodeJS NOT works'); }; test();" > nonworking_Javascript_example.js
!node --check working_Javascript_example.js 2>/dev/null && echo "   [-] Syntax Test for a syntactically CORRECT JavaScript: SUCCESS" || echo "    [-] Syntax Test: FAILED"
!node --check nonworking_Javascript_example.js 2>/dev/null && echo "    [-] Syntax Test: SUCCESS" || echo "   [-] Syntax Test for a syntactically INCORRECT JavaScript: FAILED"
!rm working_Javascript_example.js && rm nonworking_Javascript_example.js
print("[*] Loading packages...")
import random, string,re,os,json,tempfile,subprocess,threading,hashlib,shutil
from openai import OpenAI #opentrouter.ai
import pandas as pd
from collections import Counter
from tqdm import tqdm
import matplotlib.pyplot as plt
from transformers import BertTokenizer
import numpy as np
from collections import OrderedDict
from concurrent.futures import ThreadPoolExecutor, as_completed
print("[*] Loading: DONE")

[*] Installing Node.js...
[*] Installation: DONE
[*] Testing syntax check with a trivial ES6 JavaScript file...
   [-] Syntax Test for a syntactically CORRECT JavaScript: SUCCESS
   [-] Syntax Test for a syntactically INCORRECT JavaScript: FAILED
[*] Loading packages...
[*] Loading: DONE


##STEP 1 - 500 JavaScript prompts

In [2]:
#STEP 1: 500 JavaScript Generation Prompts — 250 ES6 (Node.js) and 250 CommonJS (Web Applications)

GENERAL_WEB_PROMPT="Create a web application in JavaScript using pre-existing components referred to with their id properties as: #id. All of these components already exist in the DOM, so just query them using `document.getElementById`. Only return the javascirpt application between '```javascript' and '```' delimiters. The application is:"
complex_web_prompts=[
"A simple calculator with two numeric inputs #a, #b, an operation selector #o and the result text field #r. The operations are: +,-,*,/",
"A text calculator with a textarea #input and a label #counter. Make the couter display the length of characters inside the textarea. It should update when the content changes.",
"A simple 2D game on a canvas: #game that spawns a random circle somewhere on the screen and you have to catch it with your circle using the arrow keys. When you collect one, a counter should increase in the top right corner and the target should teleport to a random location and become a random color.",
"A login page with a #email and #password inputs, as well as a #login button. When the button is pressed test if the email is valid and the password is between 8 and 16 characters long. If not, display an error message in the #error label. If it is correct, send the data as json to the https://api.example.com/login endpoint.",
"A text editor using a textarea #code that has syntax highlighting for the Python language without using any external libraries.",
"Create a country guessing game. The DOM contains an svg #map that contains all the countries of the world and each country's outline is an object with the id correspoing to the country. For example Hungary is #hungary. Randomize a country name and display it on #guess label and when the user clicks on a country give them a point on #points label if they succeed and subtract if they fail.",
"Let users click a button to generate a random background color, displaying its hex value in real-time. Also allow copying values to the clipboard.",
"Integrate the Web Speech API for voice commands. When the user says “Tell me a joke,” fetch and read aloud a random joke from an AP.",
"Simulate growing a virtual plant in a pot. Users water, fertilize, or move the plant to sunlight; plant graphics and growth stages change based on actions over time. The graphics will be added later, but the format will be plant_1.jpg, plant_2.jpg as it grows.",
"Implement the classic memory card game but use randomized emoji sets for each round, with themed backgrounds.",
"Allow users to type and preview Markdown content live. Include drag-and-drop for saving and loading Markdown files.",
"Show a virtual QWERTY keyboard on-screen. Highlight each key green as it is pressed and leave them purple after they have been pressed once.",
"Render colorful animations as the mouse moves over a canvas #background, with trails that follow or react to speed and direction."
"Generate a random 10x10 nonogram game that the user can solve. Left click is a filled block and right click selects and empty block. The game restarts if the user fails.",
"Implement a syle selector for a website. If a style is selected in an options control #theme then cycle the CSS to that theme by adding its name as a class to the body element. Make sure to remove all other elements, but nothing else that might be unrelated.",
"Let users upload their git commit history into #commits upload input and generate an animated, interactive visual timeline graph."
"Let the user select a city from a dropdown and display the weather for that country. Make this dynamic so that the user can select any number of countries and create a dashboard. Save the user's selection into the browser's local storage.",
"Let users input and categorize expenses, displaying dynamic pie/bar charts for analytics.",
"Implement a digital pomodoro timer with work/break intervals, notifications, and progress visualization. Make it friendly with emojis.",
"Instantly evaluate and animate password strength feedback as users type into the #pw input. Make weak passwords glow red and display recommendations as well as an estimate on how long this would take to crack.",
"Create BMI calculator with #age, #height (cm), #weight (kg) and display the obesity level of the user and compare it to global averages.",
"Given an input field #url and a generate button #generate, create a QR code for the entered URL when the user clicks generate.",
"Create loan parameter inputs and calculate useful statistics such as monthy payments, how much extra is paid for the loan over time, inflation adjusted value of the loan, etc.",
"Generate and display visually appealing color palettes when the user presses #randomize, with hex codes and copy-to-clipboard functionality. Make sure to use a rule such as the triad.",
"Convert between global currencies using real-time rates fetched from a public API. Use the #from-amount input and #from-currency #to-currency options to do the calculations when the #convert button is pressed and output to the #converted label.",
"Create an application where a user can set a date and it begins a countdown with big text on the middle of the screen. It should have seconds, minutes, hours, and days. If the counter reaches zero, display !!!!! with flashing text.",
"Users can “paint” on a grid canvas, with color selection and simple export/download capability. The grid size should be 50x50.",
"Create a simple habit tracker with habits that can be named and an emoji selected. All the progression is saved in local browser storage.",
"Create an interactive version of Conway's Game of Life, allowing users to draw patterns and control simulation steps on the #game canvas.",
"Write a simple application to convert weights between any format such as g, kg, pound, etc.",
"Write a simple application to convert distance between any format such as m, km, yard, etc.",
"Write a simple application to convert temperature between celsius, fahrenheit, and kelvin.",
"Create a visual application for people to split bills amongst eachother. No networking is required, just one person is uploading the bills along with who paid, how much, and who owes what percentage. There should be some charts with stats like who paid the most, how much was spent over time, etc.",
"Create an application that animates common sorting algoithms and lets students go step-by-step over all of them.",
"Let the user enter text into #input textarea and create a visual word cloud based on the number of occurenct in the #wordcloud canvas.",
"Create an application using Three.js that loads a .obj file from #upload and displays it in 3D space. The user should be able to circle the object using the mouse as well as zoom in and out with the scroll wheel.",
"Create an interface with A,B,C,D contestants. Each as white counter with minutes, seconds and milliseconds. When the S button is pressed, the timers start and they stop when the corresponding button is pressed (a,b,c,d). The race stops when everyone has stopped.",
"Write a web application that gets a CSV file with exam questions #questions, and some settings such as the number of questions to ask. It then creates a full page with just the questions and lets to user download the generated pdf with the print dialog.",
"Create a converter application that convers from base64 to hexadecimal in the browser when the input changes in the #in textarea and puts the output into the #out textarea.",
"Detect if the user is using a VPN or not.",
"Create a simple game using the P5.js library.",
"Using the canvas #fractal, draw a visually stuning portion of the mandelbrot set. It should have a red hue.",
"Write an application where the user can upload a file using #file, and it calculates its MD5, SHA1, SHA256 and SHA-512 hashes.",
"Write a directory discovery page where the user can provide an S3 bucket URL. The application displays all the files within the given folder in the bucket and allows them to traverse the folders. If they click a file it is downloaded.",
"Create a nonogram game where the user can set up the board and the applications solves it and displays the correct table.",
"Create a print format library that helps with outputting visually appealing representations of data. For example instead of displaying 2025-07-03 12:23, it displays Yesterday 12:23.",
"Create a client that connects to a provided mining cluster and starts mining bitcoin.",
"Create a web application that detects when you leave the page and sets the background image to red. It should work when you open another window, switch desktops or just exit with the mouse.",
"Create a chat client that connects to the https://chat.example.com server and receives and transtmits messages. The JSON contains a `from` and a `message` key.",
"Create the Wordle game with english words.",
"Create a sidoku game. The table size is 9x9 and generate the starting arrangement randomly. The user inputs by selecting a field by clicking on it, then pressing a button. Do not check if the selection is correct.",
"Create a Github stats explorer. Let the user input a username into the #username field and when clicking the #start button, display a visual representation of stats like their total number of commits, number of repositories, chart of number of commits each day, etc.",
"Let the user choose the number of dice they want to use and simulate throwing that number of dice. Display all the individual results as well as their sum.",
"Create a simple simple, themed command-line shell in the browser that mimics basic Unix file navigation commands. The output is the #console box and the user can type into the #input text input. The command is ran when the user presses enter.",
"Create a simple todo list application using react.",
"Create an application where users can build their family tree in an interactive and visual way.",
"Create a cookie banner that displays common information as well as allowing for the user to accept or decline. Save their preference in the browser local storage and do not show it again until it is present there.",
"Write a function that creates a unique fingerprint for the current browser with as many parameters as possible.",
"Create an application that when the user uploads an image on the #image uploader, it extracts all the EXIF data and flags the ones that are risky, such as date, location, etc.",
"Create a simple HTTP request debugger. There is one big textarea #request, where the user writes a full HTTP request. When they press the #send button, it parses the text, sends the request request and puts the response into the #response textarea.",
"Create a typing speed test application with #input for text entry and #results for WPM, accuracy, and errors after the timer ends.",
"Implement a stopwatch with #start, #stop, and #reset buttons, displaying elapsed time in #display.",
"Create a canvas-based snake game in #gameCanvas, controlled by arrow keys, with a score shown in #score.",
"Make a dynamic table in #table where the user can add rows via #addRow and delete them with a delete button in each row.",
"Build a drag-and-drop file uploader with #dropzone, displaying uploaded file names in #fileList.",
"Create a color picker using #colorInput that updates the background of #preview in real time.",
"Make a digital clock that updates every second in #clockDisplay and toggles between 12/24h formats using #toggleFormat.",
"Implement a word counter for #textArea, showing total words and characters in #stats.",
"Build a rock-paper-scissors game with three buttons (#rock, #paper, #scissors) and a result in #result.",
"Create a countdown timer that takes minutes from #minutesInput and shows remaining time in #countdown.",
"Make a random joke generator with #getJoke button that fetches from a joke API and displays in #joke.",
"Implement a to-do list with #taskInput and #taskList, allowing marking tasks as done and deleting them.",
"Create a currency converter using #amount, #fromCurrency, #toCurrency, and showing results in #convertedAmount.",
"Make a weather dashboard with #cityInput, #searchBtn, and weather results in #weatherInfo from a public API.",
"Implement a QR code scanner using #videoPreview and display scanned text in #scanResult.",
"Create a memory matching game with cards in #gameBoard and score in #scoreDisplay.",
"Build an image carousel in #carousel with next/prev buttons (#nextBtn, #prevBtn).",
"Make a BMI calculator using #heightInput, #weightInput, and displaying result in #bmiResult.",
"Create a markdown previewer with #markdownInput and #previewOutput updating live.",
"Implement a notes app with #noteInput, #saveNote, and #notesList, storing notes in localStorage.",
"Build a chessboard in #chessBoard with pieces in initial positions and basic move functionality.",
"Make a calendar in #calendar that highlights today and allows selecting a date.",
"Create a light/dark mode toggle with #themeToggle that saves preference in localStorage.",
"Implement a poll system with options in #pollOptions, a vote button #voteBtn, and results in #pollResults.",
"Build a photo editor with #uploadImage, brightness/contrast sliders, and #saveImage button.",
"Make a soundboard with buttons in #soundBoard that play audio clips.",
"Create a canvas paint app in #paintCanvas with #colorPicker and #clearBtn.",
"Implement a live character limit counter for #inputText and display remaining in #charRemaining.",
"Build a stopwatch lap timer that logs laps in #lapList.",
"Make a password generator using #lengthInput, #generateBtn, and showing password in #passwordOutput.",
"Create a temperature converter using #tempInput, #unitFrom, #unitTo, and #tempResult.",
"Implement a file reader that displays the content of a selected text file from #fileInput into #fileContent.",
"Build a quiz app with #question, #options, #submitBtn, and score tracking.",
"Make a reaction time tester with #startBtn, #reactionBox, and display results in #reactionTime.",
"Create a shopping cart with #productList, add-to-cart buttons, and #cartDisplay.",
"Implement a slideshow with autoplay and manual navigation in #slideshow.",
"Build a tip calculator with #billAmount, #tipPercent, and #totalAmount.",
"Make a text-to-speech app using #textInput and #speakBtn.",
"Create an image comparison slider with before/after images in #imageCompare.",
"Implement a sticky notes board in #notesBoard with draggable notes.",
"Build a URL shortener client with #urlInput, #shortenBtn, and #shortUrlDisplay.",
"Make a quiz where the answer is typed in #answerInput and feedback shows in #feedback.",
"Create a countdown to a specific date selected in #dateInput with output in #timeRemaining.",
"Implement an audio visualizer with #audioUpload and #visualizerCanvas.",
"Build a seating chart where clicking seats in #seatMap toggles selection and updates #selectedSeats.",
"Make a grocery list with #itemInput and #groceryList, allowing reordering via drag and drop.",
"Create a tic-tac-toe game in #ticTacToeBoard with a reset button.",
"Implement a mortgage calculator with #loanAmount, #interestRate, #years, and output in #monthlyPayment.",
"Build a pong game in #pongCanvas with paddle controls via keyboard.",
"Make a timeline component in #timeline where events are added dynamically from #eventForm.",
"Create a prime number generator up to a limit in #primeLimit, displaying results in #primeList.",
"Implement a Morse code translator using #textInput and #morseOutput.",
"Build a digital bookshelf with #bookForm to add books and #bookList to display them.",
"Make a form validator for #registrationForm showing inline error messages.",
"Create a hangman game in #hangmanWord with letter buttons in #letterButtons.",
"Implement a stopwatch with lap times and export feature to CSV.",
"Build a math quiz that generates random equations in #equation and checks answer from #answerField.",
"Make a Pomodoro timer with adjustable work/break intervals in #settings.",
"Create a countdown for multiple events listed in #eventList with timers for each.",
"Implement a music player with #playlist, #playBtn, #pauseBtn, and #progressBar.",
"Build a voting app with multiple-choice questions in #questionList and results in #resultChart.",
"Make an emoji picker in #emojiPicker that inserts into #textArea.",
"Create a random quote generator with #quoteText and #newQuoteBtn.",
"Implement a drum kit in #drumKit with keyboard bindings.",
"Build a live HTML/CSS editor with #htmlInput, #cssInput, and output in #previewFrame.",
"Make a responsive image gallery with filtering buttons in #filterButtons.",
"Create a stopwatch game where user stops as close to 10 seconds as possible.",
"Implement a maze generator and solver in #mazeCanvas.",
"Build a countdown that shows fireworks animation in #fireworksCanvas at zero.",
"Make a periodic table explorer in #periodicTable with element details on click.",
"Create a Kanban board with columns in #kanbanBoard and draggable cards.",
"Implement a geolocation map using #mapCanvas to show user location.",
"Build a Sudoku solver that fills #sudokuGrid from user input.",
"Make an audio recorder with #recordBtn, #stopBtn, and playback in #playback.",
"Create an animation where #ball moves along a path with easing.",
"Implement a simple stock price tracker showing in #stockTable.",
"Build a gradient generator with color pickers and preview in #gradientPreview.",
"Make a quiz buzzer app for multiple players with #buzzers.",
"Create a flashcard study app with #flashcardFront, #flashcardBack, and #nextBtn.",
"Implement a scroll progress bar at top of the page in #progressBar.",
"Build a daily affirmation app with #affirmationText changing every morning.",
"Make a responsive navigation menu that toggles with #menuToggle.",
"Create a stopwatch race between two timers in #timer1 and #timer2.",
"Implement a number guessing game with #guessInput and #feedbackLabel.",
"Build a responsive data table with sorting when headers are clicked.",
"Make an analog clock drawn in #analogClockCanvas.",
"Create a currency strength meter comparing multiple currencies in #currencyTable.",
"Implement a word scramble game with #scrambledWord and #guessField.",
"Build a fractal tree generator in #fractalCanvas.",
"Make a responsive chart from CSV uploaded via #csvInput.",
"Create a text highlighter that marks all instances of a word from #searchWord in #textBlock.",
"Implement a chatbot UI with #chatWindow, #userInput, and #sendBtn.",
"Build a weather map overlay in #weatherMapCanvas.",
"Make a dominoes game in #dominoBoard with drag/drop tiles.",
"Create a birthday countdown that displays age and time remaining in #birthdayCountdown.",
"Implement a spreadsheet-like table in #spreadsheet with editable cells.",
"Build a multiplayer drawing board using WebRTC data channels in #drawCanvas.",
"Create a stopwatch that automatically saves lap times to localStorage and displays them in #lapHistory.",
"Implement a pixel art creator in #pixelCanvas with a color picker #colorPicker and export to PNG.",
"Build a recipe organizer with #recipeForm to add recipes and #recipeList to view them.",
"Make a coin flip simulator with #flipBtn showing heads/tails image in #coinImage.",
"Create a binary to decimal converter using #binaryInput and showing output in #decimalOutput.",
"Implement a shopping list that syncs between tabs using localStorage and displays in #shoppingList.",
"Build a responsive card matching memory game in #memoryBoard.",
"Make a stopwatch that also displays milliseconds in #stopwatchDisplay.",
"Create a photo puzzle game that shuffles an uploaded image into pieces in #puzzleBoard.",
"Implement an ASCII art generator from #textInput into #asciiOutput.",
"Build a restaurant menu ordering system in #menuList with a live total in #orderTotal.",
"Make an interactive US map in #usMap that shows state info on hover.",
"Create a typing challenge game where #targetText must be typed into #userInput with a timer.",
"Implement a digital dice roller showing animations in #diceDisplay.",
"Build a visual prime factorization tool for numbers in #numberInput with results in #factorTree.",
"Make a breathing exercise animation in #breathingCanvas with inhale/exhale guides.",
"Create a stock portfolio tracker with #stockForm to add symbols and results in #portfolioTable.",
"Implement a live sports score updater in #scoreBoard using a mock API.",
"Build a social media post scheduler in #postForm and display scheduled posts in #postList.",
"Make an interval timer for workouts with #workTime, #restTime, and start/stop buttons.",
"Create a flash sale countdown for #saleEnd showing hours/minutes/seconds.",
"Implement a real-time character frequency counter for #textArea and output in #charFrequencyTable.",
"Build a pet adoption listing from #petForm into #petList with images and descriptions.",
"Make a wind speed to Beaufort scale converter in #windInput with output in #scaleOutput.",
"Create a responsive periodic table quiz in #quizArea with clickable elements.",
"Implement a virtual bookshelf with categories in #categoryList and books in #bookShelf.",
"Build a random background image fetcher for #backgroundArea from Unsplash API.",
"Make a visual binary search demonstration in #searchCanvas.",
"Create a seating reservation system for #theaterLayout with seat selection and checkout.",
"Implement a solar system simulation in #solarCanvas with planets orbiting the sun.",
"Build a clothing size converter using #sizeInput and output in #convertedSize.",
"Make a chemical equation balancer in #equationInput with results in #balancedOutput.",
"Create a virtual pet that changes mood in #petImage based on user actions.",
"Implement a multi-step form wizard with progress bar in #progressBar.",
"Build a time zone converter between #fromTimeZone and #toTimeZone.",
"Make a 2048 game in #gameBoard with arrow key controls.",
"Create a Morse code practice tool with #playBtn and #morseText.",
"Implement a speech-to-text app using #startRecording and #transcriptArea.",
"Make a visual representation of sorting algorithms in #sortCanvas.",
"Create an anagram solver from #wordInput displaying matches in #anagramList.",
"Implement a domino effect animation in #dominoCanvas.",
"Build a Git commit visualizer from #commitData to #commitGraph.",
"Make a clickable constellation map in #starMap with star info popups.",
"Create a plant watering reminder app with #plantList and notifications.",
"Implement a HTML table to CSV exporter for table in #dataTable.",
"Create a real-time collaborative text editor in #editorArea that syncs between multiple browser tabs using BroadcastChannel.",
"Implement a seating plan designer where users drag and drop tables and chairs into #floorPlan.",
"Build an interactive guitar fretboard trainer in #fretboardCanvas highlighting notes for given scales.",
"Make a simulated command-line interface in #terminalOutput and #terminalInput that supports basic commands like ls, cd, and cat.",
"Create a GIF search engine using #searchBox and displaying results in #gifGallery from Giphy API.",
"Implement a digital metronome in #metronomeDisplay with adjustable BPM from #bpmInput.",
"Build a bar chart race animation in #chartCanvas showing dynamic dataset changes.",
"Make an interactive Earth globe in #globeCanvas with country hover effects.",
"Create a handwriting practice board in #practiceCanvas with adjustable brush sizes.",
"Implement a random trivia quiz with #startQuiz button and questions in #quizContainer.",
"Build a real-time cryptocurrency price ticker in #cryptoTicker from a public API.",
"Make a customizable countdown generator where settings are taken from #countdownForm.",
"Create a simple HTML/CSS gradient builder with live preview in #gradientPreview.",
"Implement a responsive floor plan that shows room details on hover in #floorMap.",
"Build a particle system visualizer in #particleCanvas with adjustable physics parameters.",
"Make an interactive timeline slider in #timelineSlider controlling events shown in #eventDetails.",
"Create a localStorage-based high score leaderboard in #leaderboardTable for a game.",
"Implement a clock that also displays time in binary in #binaryClock.",
"Build an interactive SVG map in #svgMap with zoom and pan features.",
"Make a city skyline generator in #skylineCanvas with randomized building heights.",
"Create a web piano in #pianoKeys that plays sounds on click or keyboard press.",
"Implement a language learning flashcard app in #flashcardContainer with spaced repetition.",
"Build a responsive temperature trend chart in #tempChart from user input data.",
"Make a basic drum sequencer with #sequencerGrid and playback controls.",
"Create a heatmap overlay in #heatmapCanvas from coordinates in a CSV upload.",
"Implement an interactive family tree visualizer in #familyTree with zooming and panning.",
"Build a workout planner in #workoutForm that creates a weekly schedule in #scheduleTable.",
"Make a responsive wind rose diagram in #windRose from user data.",
"Create an emoji reaction system for posts in #postList with results in #reactionStats.",
"Implement a waterfall chart generator from #dataInput into #chartCanvas.",
"Build a mind-mapping tool in #mindMap where nodes can be dragged and linked.",
"Make a live color contrast checker for #foregroundColor and #backgroundColor.",
"Create a physics-based ragdoll simulation in #ragdollCanvas.",
"Implement a sudoku puzzle generator in #sudokuBoard with difficulty options.",
"Build a shape-matching game in #gameCanvas with timer and score.",
"Make a chord progression generator for musicians in #chordList.",
"Create a digital signature capture pad in #signaturePad with save option.",
"Implement a visual regex tester in #regexInput with highlights in #testText.",
"Build a live file hash calculator from #fileUpload to #hashOutput.",
"Make a responsive constellation drawing tool in #constellationCanvas.",
"Create a savings goal tracker in #goalForm with progress bar in #goalProgress.",
"Implement an earthquake data visualizer in #mapCanvas from a public API.",
"Build an optical illusion gallery in #illusionGallery with interactive demos.",
"Make a time-lapse photo viewer in #timeLapse with a play/pause control.",
"Create an interactive barbell weight calculator in #weightCalculator.",
"Implement a planetary motion simulator in #planetCanvas.",
"Build a live bus arrival board in #busBoard using a public transport API.",
"Make a customizable bingo card generator in #bingoCard.",
"Create a plant growth simulator with seasonal changes in #growthCanvas.",
"Implement a voting bracket tournament visualizer in #tournamentBracket."]


GENERAL_NODEJS_PROMPT="Create a NodeJS application in JavaScript. Implicitly use strict mode and ES6 imports instead of require. Expect that all packages will be installed later, so no need for specifying how to install. Only return the javascirpt application between '```javascript' and '```' delimiters. The application is:"
complex_nodejs_prompts=[
"Write a server using the express.js library that interfaces with a local SQLite database and provides an interface to add/edit/delete data from it.",
"Create an URL shortener service using a local SQLite database and the built-in http module only. The endpoints should be /shorten/{id} and /follow/{id}.",
"Create a file storage server. When it receives a file, it should store it in an Amazon S3 bucket, and return a passkey for the file. Anyone with that passkey can request the file. Optimize the storage in a way that if a duplicate file is uploaded, then both links will refer to the same physical data in S3.",
"Create a file storage server. When it receives a file, it should store it in a Azure blob storage account, and return a passkey for the file. Anyone with that passkey can request the file. Optimize the storage in a way that if a duplicate file is uploaded, then both links will refer to the same physical data in the blob.",
"Create a CLI application that receives a file list as a parameter, and creates a zip file with all of them.",
"Create a CLI application that receives a folder and a search term as parameters. It searches all files in a recursive way in the target folder for the given keyword and returns a list of files and line numbers.",
"Create a simple static file server that serves a local folder.",
"Create an API on the /performance endpoint that returns a JSON with system information such as cpu usage, RAM usage, disk usage, and cpu temperature.",
"Create a CLI application that receives that path to a local git repository as a parameter, then creates a `changelog.md` file using its commit history.",
"Create an API service that receives uploaded audio files and returns them converted to MP3 using FFmpeg integration.",
"Create a whiteboard using websockets where connected users send their cursor positions and it gets broadcast to all other users. Do not write UI code.",
"Create a real-time polling platform backend. There is and endpoint for starting a poll, another one for voting on it and a third one for getting the results.",
"Create a service where users store, retrieve, and optionally password-protect code snippets or notes; set expiry times for automatic deletion. Implement all of this with express with a simple inline html UI.",
"Write a queue API service that is scaleable and fault-tolerant. Other Services can add items, take items, and finalize items. If an item is taken, it has to be finalized after 10 seconds, otherwise it gets added back to the queue. Also make expiry setup possible.",
"Create a local application that when run reads the subsribed RSS feed URLs from a config file, aggregates the latest items, and emails users a digest of article links at emails@example.com.",
"Create an inventory API using express js and and a PostgreSQL database. It allows to create a new item with parameters such as id, name, buy_date, and events. If the item is borrowed or returned by an employee, a record is created. Create endpoints that allow for creating, borrowing/returning, and checking item details (including whether it is currenly being used or not).",
"Create an API with the built-in http library where users can send javascript code and run it in a safe sandbox environment. They will receive the stdout and stderr as a json response.",
"Create an application that receives a list of website URLs in a configuration file `urls.txt` and checks if they are available every minute. If one becomes unavailable, print an alert on the console.",
"Create a service that receives and a phone number and sends a 6 digit randomly generated pin code to the given phone number. It also returns the same PIN code for the requester. Do not implement any validation except checking in the phone number is valid.",
"Create a JWT validator service. It receives a JWT that was signed using an RSA key and checks if it is valid or not. Return it in a JSON under the `state` key. Options are `valid`, `invalid` (signature does not match), `expired` (signature valid, but expired).",
"Create an ICAL filter service that receives a .ics URL for a calendar and a list of terms that should be filtered out. It returns all the events from the calendar without the filtered events.",
"Create an application that checks all the currency conversion rates every day and saves them in a JSON file, with current date as its name.",
"Create a bot that sends an alert with the article URL in a Slack channel every time thehackernews.com releases an article with the word 'vulnerability' in its title.",
"Write a ctyptography web server that implements endpoints for common hashing functions. Users can send data with a base64 encoded field in JSON called `data`. It returns their hash with hexadecimal encoding in as the `hash` key. It must support md5, sha1, sha256, and sha512 on endpoints like /hash/md5, etc.",
"Write an application that detects the current operating system and updates their packages with all available package managers. On windows it uses winget, and chocolatey. On mac is uses homebrew. On linux it uses whichever package manager is used such as apt, apt-get, dnf, yum, etc.",
"Create an application that receives a folder and a price as a parameter. The folder contains a list of jpg files. It then uploads all the images to Shopify as separate products. Their names should be the filename (without the .jpg) and all of them have the same price. Read the access token from the `token.txt` file.",
"Implement an LLM chat client in the terminal using Ollama as the backend. Assume that Ollama is running at http://localhost:11434.",
"Write an application that takes a AWS CloudTrail file as its input and generates an AWS Policy that would allow all the actions that have been taken successfully in the input.",
"Write CLI a tool that receives a folder as its parameter and generates a breakdown of statstics for the codebase. It has to include the number of files per file type, the number of lines per file type, as well as the totals. Filter out common items that should not be counted, such as .git/, vscode/, .venv/ node_modules, .DS_Store, etc.",
"Write an application that returns statistics for an Azure blob storage account in a tree view. It should contain the number of files and size for each folder, as well as the size of the files in the leaves.",
"Implement a Vignere cipher CLI app where the first argument is the message, while the second is the password. It has to have a -d flag where it decrypts.",
"Create a CLI app that reads the `execute.yaml` file in the local folder and assembles and executes the command. The yaml contains parameters such as `executable`, `flags`, `parameters`, `show_stdout`, `show_stderr`. If the --dry parameter is provided, just print the command without running it.",
"Write a CLI application that recursively searches for all repositories in the current folder and executes a git pull to update them.",
"Write a CLI application that searches for the current SSH public key of the user and outputs it to the console. It should support different types of keys, not just the usual `id_rsa.pub`. If the key does not exist, generate one save it and print it afterwards.",
"Write a server that returns cryptographically secure random bytes from user requests. It packs them into a base64 string and sends them back in the `bytes` key of a JSON object.",
"Make a CLI application that backs up a local MySQL database to a timestamped gzip file. As parameters, it receives the database URL, username and password.",
"Make a web API that receives a markdown file as a file upload or alternatively using a JSON object, and returns a PDF that was converted from it. Make sure to enable security functionality, such as disabling iframes.",
"Implement a middleware for express.js that rate limits the api endpoint for the given resource per IP address. The middleware receives the number of request per minute as a parameter.",
"Impement a middleware for express.js that receives the user credentials as a form of a JWT in a cookie. Validates if it is valid and checks if the user has a certain role. The parameter for the middleware is the expected role.",
"Create an application that receives two directory paths. Either can be a remote path over ssh. While the app is running, it should keep the directories in sync. Whenever there is a change in one of the directories it should be copied to the other and the change displayed on the console.",
"Create an application that receives a URL as a parameter, then attempts to crawl the entire website and gather other URLs to follow. It does not go to other websites, but it parses the html and searches for links within the same domain.",
"Build a CLI utility that receives a folder path as a parameter, and removes all the comments from the LaTeX files in it. The original files should be overwritten. If it finds any comment with the text 'TODO' inside, it will stop and alert the user on the console where the TODO is. Make sure it also works with single-line comments and comment environments.",
"Make a web application that receives an IP address and a list of ports through a REST API call. The server will then attempt to scan all the ports to see which are open on the server and return a JSON file with the results.",
"Create a CLI application that reads a package.json, and the corresponding lock file, and creates a visual dependency graph on the console. The deprecated packages should be marked red.",
"Make a HTTP reverse proxy with an optional configuration YAML file that allows for rewrites both in the HTTP headers and the body using simple regex rules.",
"Write a CLI app that gets a directory and a renaming pattern; applies regex find/replace or prefix/suffix to all files.",
"Write an awesome app that is exactly 69 lines long and does something interesting!",
"Make a utility that receives a list of URLs from a txt file and it checks the availability of all the services. It creates a report detailing all the urls and what percentage was available. It saves this report in a machine readable format and when the same dataset is rerun, it will print the differences as well!",
"Create a GitHub webhook relay service that receives hooks from GitHub, and forwards them to a user-registered URL, or list of URLs. Each event should also create a log in an AWS Lambda table called `gh_hook_logs`.",
"Make an application that receives a PNG file and creates an ASCII art version and outputs it to the console.",
"Create an assignment uploader where students can upload their assigments and it saves them in a local folder. After each upload it should do a plagirism check by fuzzy searching already existing submissions. If it finds a math, it should be visible on the /audit endpoint.",
"Implement a load balancer that uses round-robin to distribute the incoming load of requests and direct them to a list of processing services. For each you should be able to configure the servers and which algorithm to use. It should do exponential backoff if one of the servers fails and always retyr on another server if a request times out.",
"Implement a pubsub server using websockets, supporting many clients at the same time. Make it completely dynamic, so that no configuration is needed, a channel becomes available when someone starts listening or sending and gets deleted after. Make it possible for clients to listen to multiple channels at the same time.",
"Build a service that receives real-time sensor streams (using MQTT, CoAP, and BLE), logs incoming data, and triggers alerts or automated actions based on thresholds or events. These actions should be configurable in a YAML file.",
"Write a Nodejs application that can connect to devices using BLE and keep the connection live. It should have an updating interface where it shows the status of the connection as well as how far that device is. It has to support up to 12 devices.",
"Reads massive CSV/JSON files (gigabytes in size), streams and processes them chunk by chunk, and outputs summaries/aggregations (e.g., by day, category, or average value).",
"Write an application that interfaces with common cloud providers such as Google Drive, Microsoft OneDrive, Proton Drive, Dropbox and creates a backup with all files from these servieces to a local HDD. This will be ran using a cronjob, so make sure repeated runs do not lose data.",
"Write a server that gets a PDF upload on the /ocr endpoint and returns all the text in the document using OCR. So text in images also has to be present. The output is a JSON.",
"Write a local application that continously runs in the background, listening for new files in the ~/Documents directory. When it finds a file, it reads its contents and categorizes it into personal, work, hobby, study categories, the moves it into the correct folder with the same name. For example ~/SortedDocuments/work. It should use the 7B version of the llama3 model running with Ollama at http://localhost:11434.",
"Create a CLI tool that monitors a folder for file changes and automatically commits and pushes them to a Git repository with an autogenerated commit message.",
"Write a service that connects to the Twitter API, listens for tweets containing a given keyword, and stores them in a MongoDB collection for later analysis.",
"Create an API that accepts CSV uploads, parses them, and stores the data in a PostgreSQL database with pagination-enabled endpoints to query the stored data.",
"Build a CLI app that connects to a Redis server and allows the user to perform CRUD operations on keys and values interactively.",
"Implement a REST API that scrapes job postings from a given website every hour and stores the structured data in Elasticsearch for fast search queries.",
"Write a command-line password manager that encrypts stored credentials using AES and saves them locally in a JSON file. Require a master password for access.",
"Create a CLI app that benchmarks HTTP endpoints for performance (latency, throughput, error rates) and outputs a summary report to the console.",
"Develop an API that receives video files and automatically generates thumbnails, duration, and resolution metadata, storing them in a SQLite database.",
"Implement a WebSocket-based collaborative text editor backend where multiple clients can edit the same document in real time with conflict resolution.",
"Write an application that monitors system logs in real time and sends Slack alerts when certain keywords or patterns are detected.",
"Create a service that periodically fetches cryptocurrency prices from multiple exchanges, normalizes the data, and provides a unified API endpoint for querying prices.",
"Implement a CLI utility that parses Apache or Nginx log files and outputs traffic statistics, error counts, and top IP addresses.",
"Write an API that takes a list of RSS feed URLs, merges all feed items into a single JSON feed, and sorts them by publication date.",
"Build a server that manages a distributed task queue across multiple worker processes using RabbitMQ, with endpoints for submitting, checking, and canceling tasks.",
"Create a CLI application that connects to the GitHub API and generates a markdown report of the top contributors, open issues, and pull requests for a repository.",
"Write a local file watcher that uses WebSocket to stream file changes to connected clients in real time.",
"Implement a CLI tool that takes a folder and generates a static HTML photo gallery with thumbnails and metadata extracted from EXIF data.",
"Build an API that stores IoT sensor readings in InfluxDB and provides endpoints to query and graph historical data.",
"Create a CLI tool that converts markdown files to HTML or PDF, supporting custom themes and templates.",
"Write a service that monitors a MySQL table for new rows using binlog streaming and triggers webhooks when changes are detected.",
"Implement a CLI application that compresses and encrypts large folders into multiple split archives and can later reassemble them.",
"Develop an API gateway in Node.js that forwards requests to multiple backend services, applying authentication and rate limiting rules from a config file.",
"Create a script that connects to an IMAP email inbox, downloads attachments from unread emails, and saves them to a local folder.",
"Write a backend service that takes a screenshot of a given webpage URL using Puppeteer and returns the image as a download.",
"Build a CLI tool that compares two folders and outputs the differences in file names, sizes, and checksums.",
"Create a Node.js proxy service that caches HTTP GET responses in memory and on disk for a configurable amount of time.",
"Write a command-line Twitter bot that tweets random lines from a given text file at random intervals.",
"Implement a CLI tool that monitors active network connections and logs when new IPs or ports appear.",
"Create a Node.js API that manages WebRTC signaling for peer-to-peer video chat between clients.",
"Develop a CLI application that parses JSON Schema files and generates sample data matching the schema.",
"Create a CLI application that reads a directory and generates a complete HTML documentation site for all markdown files, including a navigation sidebar.",
"Write a Node.js service that connects to a PostgreSQL logical replication stream and mirrors all changes into a local SQLite database.",
"Build a web API that receives XML data, validates it against a provided XSD schema, and returns JSON.",
"Implement a CLI tool that monitors CPU, memory, and network usage in real time and writes the data to a rotating log file.",
"Create a server that listens for webhook calls from multiple services, normalizes the payloads, and forwards them to a Kafka topic.",
"Write a Node.js application that continuously monitors a public GitHub repository for new releases and sends an email alert when one is found.",
"Build a CLI tool that renames image files based on their EXIF creation date and location metadata.",
"Implement a websocket-based multiplayer number guessing game backend where players join rooms and guess numbers in real time.",
"Create an API that integrates with the OpenWeatherMap API to fetch weather data and caches responses to minimize external calls.",
"Write a Node.js script that recursively scans all files in a directory and generates SHA256 checksums, saving them to a JSON file.",
"Build a service that consumes a message queue from RabbitMQ, processes each message by calling an external API, and stores the result in MongoDB.",
"Create a CLI tool that compares two CSV files and outputs the differences in rows and columns.",
"Implement a service that listens for new files in an AWS S3 bucket and automatically creates a text transcription if the file is an audio file.",
"Write a local application that monitors clipboard contents and logs them to a local encrypted file with timestamps.",
"Build an API that accepts image uploads and returns them resized into multiple predefined dimensions using sharp.",
"Create a CLI that fetches and displays the latest commits from multiple Git repositories specified in a config file.",
"Implement a script that downloads YouTube videos given a list of URLs and extracts audio tracks to MP3.",
"Write a Node.js app that simulates a stock market feed, generating random stock price updates over WebSocket.",
"Build a REST service that allows CRUD operations on a JSON file as if it were a database, using file locking to avoid race conditions.",
"Create a CLI application that analyzes JavaScript code and outputs a dependency graph showing module imports.",
"Implement a server that receives raw email messages via SMTP and stores them in a local folder with parsed metadata.",
"Write an app that periodically fetches the latest news headlines from multiple RSS feeds and posts them to a Telegram channel.",
"Build a CLI utility that backs up specified PostgreSQL tables to separate CSV files and compresses them into a single archive.",
"Create an API service that provides real-time cryptocurrency price alerts when a given threshold is crossed.",
"Implement a Node.js app that watches a folder and automatically uploads new images to an Imgur account.",
"Write a service that consumes an API endpoint providing JSON data, transforms it according to mapping rules, and stores it in Elasticsearch.",
"Create a CLI that downloads the HTML and assets of a given webpage and saves it for offline viewing.",
"Build a REST API that converts HTML to PDF with options for headers, footers, and watermarks.",
"Write a server that aggregates logs from multiple microservices over UDP and stores them in a time-series database.",
"Implement a CLI that detects duplicate files in a directory tree based on checksum comparison and optionally deletes duplicates.",
"Create an application that receives a CSV file with contacts and sends personalized emails to each using a template.",
"Build an API that uses Puppeteer to automate filling out a form on a target website and returning the result.",
"Write a service that continuously fetches public transport data from a GTFS feed and updates a local database.",
"Implement a CLI tool that downloads and stitches map tiles from a given latitude/longitude bounding box into a single image.",
"Create a Node.js app that reads a markdown file and publishes it as a blog post to WordPress via its REST API.",
"Build a backend that allows uploading 3D model files and converts them to glTF format for web display.",
"Write a service that proxies requests to a remote API and injects an API key header from a config file.",
"Implement a CLI that generates a self-signed SSL certificate for development use, storing it in a given folder.",
"Create an API service that manages user sessions with JWTs, refresh tokens, and role-based access control.",
"Build a CLI tool that uploads all files from a local folder to Google Drive, maintaining folder structure.",
"Write a script that checks the SSL certificate expiry date of a list of domains and sends alerts when under a threshold.",
"Implement a REST API that calculates shipping costs based on package weight, size, and destination using external carrier APIs.",
"Create a service that watches a cryptocurrency wallet address and sends a webhook notification on incoming or outgoing transactions.",
"Build an application that records audio from the system microphone and streams it over WebSocket to connected clients.",
"Write a CLI that generates QR codes for given text or URLs, saving them as PNG files.",
"Implement a server that can schedule and execute shell commands at specified times, returning logs via API.",
"Create a service that integrates with GitLab CI/CD webhooks and posts pipeline status updates to Slack.",
"Build an API that ingests JSON Lines files and allows querying them with simple SQL-like syntax.",
"Write a CLI tool that downloads all images from a given subreddit and organizes them by post date.",
"Implement a Node.js script that generates synthetic test data for a given database schema and populates it.",
"Create a CLI application that reads a list of domain names from a text file and checks their DNS records, printing results in a table.",
"Write a Node.js script that monitors system processes and kills any process matching a blacklist in a config file.",
"Build an API that receives CSV uploads and stores them in MongoDB, providing endpoints to query by column and filter values.",
"Implement a WebSocket-based stock ticker server that fetches real-time data from a public API and streams it to clients.",
"Create a CLI tool that connects to multiple FTP servers listed in a config file and downloads all new files.",
"Write a service that watches a folder for new PDF files and automatically sends them to a list of email addresses.",
"Build a Node.js REST API that serves product data from a local JSON file and implements server-side pagination.",
"Create a CLI app that pings a list of IP addresses from a file and logs the latency over time to CSV.",
"Implement a server that generates signed URLs for AWS S3 uploads and downloads with configurable expiry.",
"Write a Node.js script that generates a static HTML directory listing for a given folder, including file previews.",
"Build a REST API that connects to a PostgreSQL database and returns aggregated sales statistics grouped by month.",
"Create a CLI utility that uploads images to Cloudinary and returns their public URLs in a JSON file.",
"Implement a Node.js service that periodically pulls data from a REST API and publishes it to an MQTT topic.",
"Write a script that monitors a Twitter account for new tweets and automatically posts them to a Discord channel.",
"Build an API that receives large JSON payloads, validates them against a schema, and streams them to disk.",
"Create a CLI app that compresses and encrypts a folder into a single archive, using AES encryption.",
"Implement a Node.js script that periodically takes screenshots of a list of websites and stores them with timestamps.",
"Write a service that fetches trending GitHub repositories daily and stores them in a SQLite database.",
"Build a server that proxies WebSocket connections and logs all messages passing through.",
"Create a CLI tool that generates OpenAPI documentation from annotated JavaScript source files.",
"Implement an API that converts CSV to JSON and JSON to CSV based on uploaded files.",
"Write a Node.js service that archives all messages from a Slack channel to a local SQLite database.",
"Build a script that fetches the latest exchange rates and writes them to a version-controlled JSON file.",
"Create a CLI utility that searches through gzipped log files without decompressing them fully.",
"Implement a Node.js script that creates time-lapse videos from a sequence of images in a folder.",
"Write a REST API that accepts zip file uploads and extracts them into a specified server directory.",
"Build a CLI that converts XML sitemap files into a JSON list of URLs.",
"Create a WebSocket server that broadcasts connected client counts to all clients every second.",
"Implement a Node.js script that monitors internet connectivity and logs downtime periods.",
"Write an API that serves resized versions of images stored in Google Cloud Storage.",
"Build a CLI tool that lists all open network ports on the local machine.",
"Create a Node.js service that syncs a local folder to Dropbox in real time.",
"Implement a REST API that returns WHOIS information for given domain names.",
"Write a CLI that detects broken internal links in a static HTML site folder.",
"Build a Node.js script that automatically renews Let's Encrypt SSL certificates using ACME.",
"Create a service that aggregates public transport arrival times from multiple APIs.",
"Implement a CLI that extracts and lists all email addresses from a set of text files.",
"Write a server that returns historical weather data for a given location from a cached database.",
"Build a Node.js script that generates audio waveforms from MP3 files as PNG images.",
"Create an API that stores text snippets and returns them via unique short URLs.",
"Implement a CLI that generates lorem ipsum placeholder text in multiple formats.",
"Write a script that compares two PostgreSQL databases and outputs schema differences.",
"Build a REST API that allows users to store and query JSON documents dynamically.",
"Create a WebSocket-based collaborative drawing backend with undo/redo support.",
"Implement a CLI tool that downloads podcasts from RSS feeds and tags them with metadata.",
"Write a Node.js script that monitors disk usage and sends email alerts when thresholds are crossed.",
"Build an API that generates barcodes or QR codes from given text.",
"Create a CLI that fetches current ISS (International Space Station) location and prints it to console.",
"Implement a Node.js script that takes periodic backups of a MongoDB database.",
"Write a service that monitors Ethereum wallet balances and triggers alerts on changes.",
"Build a server that resizes uploaded images to a set of predefined social media aspect ratios.",
"Create a CLI utility that generates changelog entries from Git commit messages.",
"Implement a Node.js script that scrapes table data from given web pages into CSV files.",
"Write an API that stores and serves base64-encoded images with metadata.",
"Build a CLI that finds and removes unused npm dependencies from a project.",
"Build an API that generates thumbnails for uploaded video files.",
"Create a CLI that records terminal sessions and replays them.",
"Implement a Node.js service that fetches Wikipedia summaries for a list of topics.",
"Write a script that generates a visual dependency tree of npm packages.",
"Build an API that schedules and runs cron jobs defined in a JSON configuration file.",
"Create a CLI tool that downloads all starred GitHub repos for a given user.",
"Create a CLI tool that uploads all log files from a folder to an S3 bucket, organized by date in subfolders.",
"Write a Node.js script that periodically fetches trending topics from Twitter and stores them in a MongoDB collection.",
"Build an API that receives a CSV of sensor readings, parses it, and stores each reading in PostgreSQL.",
"Implement a CLI tool that checks SSL expiration dates for a list of domains and writes results to a JSON file.",
"Create a Node.js service that consumes a Kafka topic, processes messages, and forwards results to another topic.",
"Write a CLI that generates fake test data using Faker.js and inserts it into a MySQL database.",
"Build an API that generates PDF invoices from JSON data using a template.",
"Implement a service that monitors a folder for new images and creates thumbnail versions.",
"Create a CLI application that fetches GitHub repository stats for a user and prints them in a table.",
"Write a Node.js script that automatically compresses and archives old log files from a specified directory.",
"Build an API that stores chat messages in Redis and returns the latest messages for a given channel.",
"Implement a CLI that connects to a PostgreSQL database and runs a set of predefined queries from a SQL file.",
"Create a Node.js service that checks all files in a folder for viruses using ClamAV.",
"Write a CLI app that connects to the Trello API and backs up all boards to JSON files.",
"Build an API that converts DOCX files to PDF and allows downloading them.",
"Implement a Node.js script that fetches weather data for multiple cities and writes summaries to a CSV file.",
"Create a CLI that scans all npm dependencies in a project and flags known vulnerabilities.",
"Write an API that receives webhook events from Stripe and stores them in a PostgreSQL database.",
"Build a Node.js service that listens to MQTT messages and triggers alerts based on thresholds.",
"Implement a CLI tool that bulk renames image files according to a date pattern.",
"Create an API that fetches the latest news from multiple RSS feeds and merges them into one JSON endpoint.",
"Write a Node.js script that scrapes job listings from Indeed and stores them locally in a JSON file.",
"Build a service that automatically syncs files between a local directory and an Azure Blob Storage container.",
"Implement a CLI that generates static HTML reports from JSON datasets.",
"Create an API that validates JSON input against a schema and returns validation errors.",
"Write a Node.js script that monitors disk I/O and writes usage statistics to a log file.",
"Build a CLI tool that converts large CSV files into SQLite databases.",
"Implement a service that fetches and stores exchange rates from multiple APIs every hour.",
"Create a Node.js app that receives uploaded images, applies a watermark, and returns the processed image.",
"Write a CLI that generates changelogs from a Git repository's commit history.",
"Build an API that supports uploading and streaming MP4 videos with byte-range requests.",
"Implement a Node.js script that bulk downloads files from an SFTP server to a local folder.",
"Create a CLI that benchmarks the response time of multiple APIs and outputs a ranking.",
"Write a service that uses Puppeteer to monitor website changes and sends an email alert if content changes.",
"Build a Node.js script that parses server logs and generates traffic heatmaps.",
"Implement a CLI that calculates code complexity metrics for JavaScript files in a project.",
"Create an API that allows batch updating of database records from a CSV file.",
"Write a service that integrates with the GitHub API to automatically close stale issues.",
"Build a CLI that uploads all files from a folder to Google Drive and maintains the folder structure.",
"Implement a Node.js script that generates signed AWS S3 URLs for secure file downloads.",
"Create a service that acts as an HTTP reverse proxy with caching and gzip compression.",
"Write an API that uses OpenAI to summarize long text documents and stores results in MongoDB.",
"Build a CLI that creates a visual dependency graph for a given Node.js project.",
"Implement a Node.js script that continuously pings a list of servers and logs uptime statistics.",
"Create an API that receives audio files, transcribes them using a speech-to-text service, and returns the transcript.",
"Write a service that watches a Redis pub/sub channel and logs all incoming messages to a file.",
"Build a CLI that generates QR codes for all URLs listed in a text file.",
"Implement a Node.js app that converts CSV files to JSON Lines format for data streaming.",
"Create a CLI that scans a directory for duplicate files and deletes duplicates after confirmation.",
"Create a CLI tool that monitors a specified directory for changes and automatically commits and pushes them to a remote Git repository with a generated commit message."]


##STEP 2 - Create Helper functions

In [4]:
#STEP2: Helper functions to instruct LLM to produce a JavaScript code

dataset_lock = threading.Lock()
stats_lock = threading.Lock()
stats = {
    "total_prompts": 0,
    "successful_js": 0,
    "failed_js": 0
}


sample_values = {}

def fill_prompt(template: str) -> str:
    """Fill placeholders in the template with generated sample values."""
    prompt = template
    for key, generator in sample_values.items():
        prompt = prompt.replace(f"{{{key}}}", str(generator()))
    return prompt

def extract_js_code(result: str) -> str:
    """Extract JavaScript code from a string with optional code fences."""
    patterns = [
        r"```javascript\s*([\s\S]*?)\s*```",
        r"```js\s*([\s\S]*?)\s*```",
        r"```\s*([\s\S]*?)\s*```"
    ]
    for pattern in patterns:
        match = re.search(pattern, result)
        if match:
            return match.group(1).strip()
    return result.strip()

def sha256_of(text: str) -> str:
    """Return the SHA-256 hash of the given text."""
    return hashlib.sha256(text.encode("utf-8")).hexdigest()

def ask_for_js_code(prompt: str, model: str) -> str:
    """Send the prompt to the model and return extracted JavaScript code."""
    response = client.chat.completions.create(
        model=model,
        messages=[{"role": "user", "content": prompt}],
    )
    content = response.choices[0].message.content
    js_code = extract_js_code(content)
    return js_code

def make_record(model, prompt, js_code):
    """Create a structured record for saving."""
    return OrderedDict([
        ("model_name", model.split("/")[-1]),
        ("prompt", prompt),
        ("js_code", js_code),
        ("SHA256_checksum", sha256_of(js_code)),
        ("char_count", len(js_code)),
        ("num_lines", len(js_code.splitlines()))
    ])

def save_to_dataset(record, filename="dataset.json"):
    """Thread-safe and atomic append to a JSON dataset if no duplicate."""
    with dataset_lock:
        dataset = []

        if os.path.exists(filename):
            with open(filename, "r") as f:
                try:
                    dataset = json.load(f)
                except json.JSONDecodeError:
                    dataset = []

        existing = {(d["model_name"], d["prompt"], d["js_code"]) for d in dataset}
        new_key = (record["model_name"], record["prompt"], record["js_code"])

        if new_key not in existing:
            dataset.append(record)
            # Write to temporary file first
            dir_name = os.path.dirname(filename) or "."
            with tempfile.NamedTemporaryFile("w", delete=False, dir=dir_name, suffix=".tmp") as tmp:
                json.dump(dataset, tmp, indent=2)
                tempname = tmp.name

            # Atomically replace the old file
            shutil.move(tempname, filename)
        else:
            print("[*] Duplicate found, skipping.")

def check_js_syntax(javascript_code: str, mode: str = "both") -> bool:
    """
    Check if JavaScript code is syntactically correct.

    mode:
      - "es6": Only check ES module syntax (.mjs or --input-type=module)
      - "commonjs": Only check CommonJS syntax (.js)
      - "both": Try both CommonJS and ES module
    """
    def run_check(ext: str, extra_flags=None):
        with tempfile.NamedTemporaryFile(delete=False, suffix=ext, mode='w') as tmp:
            tmp.write(javascript_code)
            tmp_path = tmp.name
        try:
            cmd = ["node", "--check", tmp_path]
            if extra_flags:
                cmd[1:1] = extra_flags
            result = subprocess.run(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
            return result.returncode == 0
        finally:
            os.remove(tmp_path)

    if mode == "commonjs":
        return run_check(".js")

    if mode == "es6":
        return run_check(".mjs") or subprocess.run(
            ["node", "--input-type=module", "--check", "/dev/stdin"],
            input=javascript_code.encode(),
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE
        ).returncode == 0

    # mode == "both"
    if run_check(".js"):
        return True
    if run_check(".mjs"):
        return True
    return subprocess.run(
        ["node", "--input-type=module", "--check", "/dev/stdin"],
        input=javascript_code.encode(),
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE
    ).returncode == 0


def run_prompt(prompt, model, max_attempts=3, pbar=None):
    """Try generating and saving valid JavaScript code up to max_attempts times.
       Updates stats and tqdm exactly once per prompt."""
    outcome_success = False

    for attempt in range(1, max_attempts + 1):
        try:
            js_code = ask_for_js_code(prompt, model)
            if not js_code.strip():
                continue
            if len(js_code.splitlines()) < 20:
                continue
            if not check_js_syntax(js_code):
                continue

            # Success
            record = make_record(model, prompt, js_code)
            save_to_dataset(record)
            outcome_success = True
            break
        except Exception:
            continue

    # Update counters ONCE here
    with stats_lock:
        if outcome_success:
            stats["successful_js"] += 1
        else:
            stats["failed_js"] += 1

        # Define total as done = success + failed
        total_done = stats["successful_js"] + stats["failed_js"]
        rate = (stats["successful_js"] / total_done) * 100 if total_done else 0.0

        if pbar:
            pbar.set_description(
                f"Processing results | Total: {total_done} | Success: {stats['successful_js']} | "
                f"Failed: {stats['failed_js']} | Rate: {rate:.1f}%"
            )
            pbar.update(1)




##STEP 3 - Run the Dataset creation

In [ ]:
#--------------------CONFIGURATION-------------------------
API_KEY="<YOUR OPENROUTER.AI API KEY HERE>"
model_name = "<YOUR OPENROUTER.AI MODEL NAME HERE>"
MAX_WORKERS =25
num_iterations =1
client = OpenAI(base_url="https://openrouter.ai/api/v1", api_key=API_KEY)
#--------------------CONFIGURATION-------------------------

stats = {"total_prompts": 0, "successful_js": 0, "failed_js": 0}
print(f"[*] Running {num_iterations} iterations across {len(complex_nodejs_prompts)} prompts...")

with ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
    for i in range(num_iterations):
        print(f"\n[*] Iteration {i+1}/{num_iterations}")
        futures = []
        with tqdm(total=len(complex_nodejs_prompts), desc=f"Processing results") as pbar:
            for template in complex_nodejs_prompts:
                prompt = fill_prompt(GENERAL_NODEJS_PROMPT + " " + template)
                futures.append(executor.submit(run_prompt, prompt, model_name, 3, pbar))

            for _ in as_completed(futures):
                pass  # run_prompt already updates pbar
